<a href="https://colab.research.google.com/github/chaitu2568/NLP_INTENT_CLASSIFICATION/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Mar 26 17:15:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    24W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install tensorflow-gpu >> /dev/null

ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.


In [3]:
!pip install --upgrade grpcio >> /dev/null

ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement grpcio~=1.24.3, but you'll have grpcio 1.27.2 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.


In [0]:
!pip install tqdm  >> /dev/null

In [0]:
!pip install bert-for-tf2 >> /dev/null

In [0]:
!pip install sentencepiece >> /dev/null

In [7]:
import os
import math
import datetime
import json
import requests

from tqdm import tqdm

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import seaborn as sns
from pylab import rcParams

from sklearn.metrics import confusion_matrix, classification_report


RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

In [0]:
df = json.loads(requests.get('https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_small.json').text)

In [0]:

''' Getting Training data '''
inscope_train = pd.DataFrame(df['train'],columns=['query','intent'])
oss_train = pd.DataFrame(df['oos_train'],columns=['query','intent'])

''' Getting Validation data '''
inscope_val = pd.DataFrame(df['val'],columns=['query','intent'])
oss_val = pd.DataFrame(df['oos_val'],columns=['query','intent'])

''' Getting Test data '''
inscope_test = pd.DataFrame(df['test'],columns=['query','intent'])
oss_test = pd.DataFrame(df['oos_test'],columns=['query','intent'])


train_df = pd.concat([inscope_train,oss_train])
val_df = pd.concat([inscope_val,oss_val])
test_df = pd.concat([inscope_test,oss_test])


In [10]:
train_df.shape

(7600, 2)

In [0]:
train_df = train_df.append(val_df).reset_index(drop=True)

In [12]:
train_df.shape

(10700, 2)

In [13]:
train_df.head()

,query,intent
0,can you walk me through setting up direct depo...,direct_deposit
1,i want to switch to direct deposit,direct_deposit
2,set up direct deposit for me,direct_deposit
3,how do i go about setting up direct deposit,direct_deposit
4,i need to get my paycheck direct deposited to ...,direct_deposit


In [14]:
 !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-03-26 17:17:09--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.112, 2607:f8b0:4007:80e::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   122MB/s    in 3.2s    

2020-03-26 17:17:13 (122 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [15]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
os.makedirs("model", exist_ok=True)

In [0]:
!mv uncased_L-12_H-768_A-12/ model

In [0]:
bert_model_name="uncased_L-12_H-768_A-12"
bert_check_point_dir = os.path.join("model/", bert_model_name)
bert_check_point_file = os.path.join(bert_check_point_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_check_point_dir, "bert_config.json")

In [0]:
class IntentClassificationTask:
  DATA_COLUMN = "query"
  LABEL_COLUMN = "intent"

  def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
    self.tokenizer = tokenizer
    self.max_seq_len = 0
    self.classes = classes
    
    ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._split_data, [train, test])

    print("maximum seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)
    self.train_x, self.test_x = map(self._zeropadding, [self.train_x, self.test_x])

  def _split_data(self, df):
    x, y = [], []
    
    for _, row in tqdm(df.iterrows()):
      text, label = row[IntentClassificationTask.DATA_COLUMN], row[IntentClassificationTask.LABEL_COLUMN]
      # It tokenize the given Sequence
      tokens = self.tokenizer.tokenize(text)
      # Adding the starter and ending for each sequence
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      #Convert the tokens from text to vector format
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)

      self.max_seq_len = max(self.max_seq_len, len(token_ids))
      x.append(token_ids)
      # Using the index of labels
      y.append(self.classes.index(label))

    return np.array(x), np.array(y)

  def _zeropadding(self, ids):
    x = []
    for input_ids in ids:
      input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)


In [0]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_check_point_dir, "vocab.txt"))

In [0]:
def create_model(max_seq_len, bert_check_point_file):

  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
    #Reading from the BERT Config File
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert-large-uncased")
        
  input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
  bert_output = bert(input_ids)

  print("bert shape", bert_output.shape)

  # Output Layer to input BERT Layer
  cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
  # Adding Drop-out Layer
  cls_out = keras.layers.Dropout(0.05)(cls_out)
  # Adding Dense output Layer to drop-out Layer (using 'tanh' activation function)
  logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
  # Adding Drop-out Layer
  logits = keras.layers.Dropout(0.05)(logits)
  # Adding Dense output Layer to drop-out Layer (using 'softmax' activation function)
  logits = keras.layers.Dense(units=len(classes), activation="softmax")(logits)
  
  model = keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_check_point_file)
        
  return model

In [22]:
classes = train_df.intent.unique().tolist()

data = IntentClassificationTask(train_df, test_df, tokenizer, classes, max_seq_len=128)

10700it [00:02, 4001.42it/s]
5500it [00:01, 3944.01it/s]


maximum seq_len 32


In [23]:
model = create_model(data.max_seq_len, bert_check_point_file)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
bert shape (?, 32, 768)
Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f4eab5c3eb8> (prefix:bert-large-uncased). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [24]:
data.train_x.shape

(10700, 32)

In [25]:
data.train_x[5]

array([  101,  2054,  2024,  1996,  4084,  2000,  2275,  2039,  3622,
       12816,  2000,  2026,  5252,  4070,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0])

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 32)]              0         
_________________________________________________________________
bert-large-uncased (BertMode (None, 32, 768)           108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_38 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 151)               116119

In [0]:
model.compile(
  optimizer=keras.optimizers.Adam(2e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

In [28]:
log_dir = "log/intent_classification/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
  x=data.train_x, 
  y=data.train_y,
  validation_split=0.1,
  batch_size=32,
  shuffle=True,
  epochs=20,
  callbacks=[tensorboard_callback]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 9630 samples, validate on 1070 samples
Epoch 1/20
9630/9630 [==============================] - 112s 12ms/sample - loss: 5.0046 - acc: 0.0241 - val_loss: 5.0072 - val_acc: 0.0206
Epoch 2/20
9630/9630 [==============================] - 104s 11ms/sample - loss: 4.7574 - acc: 0.2916 - val_loss: 4.9392 - val_acc: 0.0935
Epoch 3/20
9630/9630 [==============================] - 105s 11ms/sample - loss: 4.4982 - acc: 0.5533 - val_loss: 4.6896 - val_acc: 0.3664
Epoch 4/20
9630/9630 [==============================] - 105s 11ms/sample - loss: 4.3357 - acc: 0.7102 - val_loss: 4.5721 - val_acc: 0.4664
Epoch 5/20
9630/9630 [==============================] - 104s 11ms/sample - loss: 4.2498 - acc: 0.7918 - val_loss: 4.4640 - val_acc: 0.5748
Epoch 6/20
9630/9630 [==============================] - 104s 11ms/sample - loss: 4.1876 - acc: 0.8513 - val_loss: 4.3970 - val_acc: 0.6402
Epoch 7/20
9630/9630 [==

In [29]:
_, train_accuracy = model.evaluate(data.train_x, data.train_y)
_, test_accuracy = model.evaluate(data.test_x, data.test_y)

print("training accuracy", train_accuracy)
print("test accuracy", test_accuracy)

5500/5500 [==============================] - 21s 4ms/sample - loss: 4.1726 - acc: 0.8667
training accuracy 0.98850465
test accuracy 0.8667273
